In [82]:
from django_for_jupyter import init_django
init_django()

In [2]:
import pandas as pd
import numpy as np
from django.db.models import F, Sum, Count
from precios.models import SiteURLResults, Articulos, Unifica, Marcas, Vendedores, MarcasSistema, ReemplazaPalabras
from members.models import DetalleLista
from precios.pi_get import fixMarca, obtener_grados, get_palabras_con_numychar
from precios.pi_functions import createRule, buscar_articulos_con_y_sin_grados
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from functools import partial

In [83]:
# def find_duplicates(row):
#     articulo = row['nombre']
#     medida_cant = row['medida_cant']
#     unidades = row['unidades']
#     marca = row['marca']
#     grados = row['grados']
    
#     df_matches = df_articulos.loc[
#         (df_articulos['marca'] == marca) & 
#         (df_articulos['medida_cant'] == medida_cant) & 
#         (df_articulos['unidades'] == unidades) & 
# #         (~df_articulos['ean_13'].isnull()) & 
#         (df_articulos['nombre'] != articulo)
#     ]
    
#     max_ratio = 0
#     max_match = None
    
#     for _, match in df_matches.iterrows():
#         ratio = fuzz.token_set_ratio(articulo.lower(), match['nombre'].lower())
#         if ratio > max_ratio:
#             max_ratio = ratio
#             max_match = match
    
#     if max_ratio >= 90:
#         return {
#             'marca': marca,
#             'medida_cant': medida_cant,
#             'unidades': unidades,
#             'articulo_con_grados': articulo,
#             'ean_13_con_grados': row['ean_13'],
#             'articulo_sin_grados': max_match['nombre'],
#             'ean_13_sin_grados': max_match['ean_13'],
#         }
    
#     return None

In [5]:
# Crear una función para comparar las medidas y unidades de dos artículos
def compare_measure(medida_cant1, medida_cant2, unidades1, unidades2):
    return fuzz.ratio(medida_cant1 + unidades1, medida_cant2 + unidades2)

In [8]:
# querySiteArticulos = Articulos.objects.all()
querySiteArticulos = Articulos.objects.annotate(dcount=Count('vendedores__vendidoen')).all()

#df_articulos = pd.DataFrame(list(querySiteArticulos.values('nombre','marca','medida_cant', 'unidades', 'grados', 'ean_13')))
df_articulos = pd.DataFrame(list(querySiteArticulos.values('pk','nombre','marca','medida_cant', 'unidades', 'grados', 'ean_13', 'dcount')))



In [9]:
# reemplazamos los valores vacíos en el DataFrame con None
df_articulos = df_articulos.replace(r'^\s*$', value=None, regex=True)

In [10]:
# filtramos los artículos que tienen valores no nulos en la columna "grados"
df_filtrado = df_articulos[df_articulos['grados'].notnull()]

# obtenemos las marcas únicas en el DataFrame filtrado
marcas_con_grados = df_filtrado['marca'].drop_duplicates().tolist()



In [11]:


def buscar_articulos_2(df):
    # Ordenar el dataframe por dcount en orden descendente
    df = df.sort_values(by=['dcount'], ascending=False)

    # Crear una lista de nombres de artículo únicos
    nombres_articulos_unicos = df['nombre'].unique()

    # Crear una lista de posibles nombres de artículo que suenan igual
    posibles_nombres = []

    # Buscar nombres de artículo que suenan igual
    nombres_comparados = [] # lista para almacenar los nombres ya comparados
    for i, nombre in enumerate(nombres_articulos_unicos):
        if nombre not in nombres_comparados: # verificar si el nombre ya ha sido comparado
            # buscar los nombres de artículo que tienen una puntuación de coincidencia de al menos el 90% y una diferencia de dcount menor o igual a 10
            nombres_coincidentes = []
            for j in range(i+1, len(nombres_articulos_unicos)):
                if nombres_articulos_unicos[j] not in nombres_comparados:
                    if fuzz.token_sort_ratio(nombre, nombres_articulos_unicos[j]) >= 90 and abs(df[df['nombre']==nombre]['dcount'].iloc[0] - df[df['nombre']==nombres_articulos_unicos[j]]['dcount'].iloc[0]) <= 10:
                        nombres_coincidentes.append(nombres_articulos_unicos[j])
            if nombres_coincidentes:
                # agregar el nombre de artículo y su lista de nombres de artículo coincidentes a la lista de posibles nombres
                posibles_nombres.append((nombre, nombres_coincidentes)) 
                # agregar el nombre actual a la lista de nombres comparados
                nombres_comparados.append(nombre)
                # agregar los nombres coincidentes a la lista de nombres comparados
                nombres_comparados += nombres_coincidentes

    # mostrar la lista de posibles nombres
    return posibles_nombres

In [17]:
cuenta = 0
for marca in marcas_con_grados:
    cuenta = cuenta + 1
    mmmarca = Marcas.objects.filter(id=marca).get()
#     lista_vendedores = mmmarca.vendedoresList
#     if len(lista_vendedores) > 1:
        
        
    df_nueva_marca = df_articulos[(df_articulos['marca'] == marca) ]
        

    lista = buscar_articulos_2(df_nueva_marca)
    print("en if")
#         print(lista)
#         for item in lista:
#             for si_nompaso in item[1]:
#                 si_marca, entonces_marca = marca, marca
#                 si_nombre = si_nompaso
#                 entonces_nombre = item[0]

#                 print(f'Marca={mmmarca} si_nombre={si_nombre} entonces_nombre={entonces_nombre}')
                
#                 respuesta = input("Crea Regla ? (s/N)...")
#                 print(respuesta)

    if cuenta > 2:
        break
# #     df_duplicates = pd.DataFrame([r for r in results if r is not None])
# #     df_duplicates
#          )
print(cuenta)

en if
en if
en if
3


In [14]:

####     df_nueva_marca_sin_grados = buscar_articulos_sin_grados(df_nueva_marca)
#     df_combinado = buscar_articulos_con_y_sin_grados(df_nueva_marca)
#     for articulo in df_combinado:
#         si_marca       = Marcas.objects.filter(id=articulo['si_marca']).get()
#         entonces_marca = Marcas.objects.filter(id=articulo['entonces_marca']).get()
#         createRule(
#             si_marca, 
#             articulo['si_nombre'], 
#             articulo['si_grados'], 
#             articulo['si_medida_cant'], 
#             articulo['si_unidades'], 
#             entonces_marca, 
#             articulo['entonces_nombre'], 
#             articulo['entonces_grados'], 
#             articulo['entonces_medida_cant'], 
#             articulo['entonces_unidades'] 




nombre,  grados = obtener_grados(nombre)

print(nombre, grados)

pasta fettuccine 12 ° None


In [81]:
import re
# import operator as op
# # define all digits
# all_digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# # define all letters
# all_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# def separate_number_chars(s):
#     res = re.split('([-+]?\d+\.\d+)|([-+]?\d+)', s.strip())
#     res_f = [r.strip() for r in res if r is not None and r.strip() != '']
#     return res_f



nombre = 'Gelatina En Polvo Arándano Mora Daily, 22.5g 22,5g 15,0° '
nombre = 'Coctel Secreto Peruan Zero Lim 700cc 15g '
nombre = 'Pepsi 3lt Desechable'
print(nombre)
res = get_palabras_con_numychar(nombre)


for palabra in res:
    palas = re.split('([A-Za-z]+[\d@]+[\w@]*|[\d@]+[A-Za-z]+[\w@]*)', palabra.strip())
    
    # print('palas=',palas)
    # palas
    reemplaza_con = ''
    for pa in palas:
        if pa != '':
            if len(get_palabras_con_numychar(pa)) > 0 :
                digitos = re.split('(\d+)', pa.strip())
                for digito in digitos:
                    if digito != '':
                        reemplaza_con = reemplaza_con + digito + ' '
                        print(f'digito={digito}')
            else:
                print(f'pa={pa}')
                reemplaza_con = reemplaza_con + pa 

    print(f'palabra={palabra} reemplaza_con={reemplaza_con}' )
    nombre = nombre.replace(palabra, reemplaza_con) 
        

print(nombre)

Pepsi 3lt Desechable
['3lt']
['3lt']
digito=3
digito=lt
palabra=3lt reemplaza_con=3 lt 
Pepsi 3 lt  Desechable
